In [1]:
!pip install -r requirements.txt
!pip install interpolation 
from interpolation.splines import UCGrid, eval_linear  # see, https://github.com/EconForge/interpolation.py
import numpy as np
from numba import jit, njit
import statsmodels.api as sm
from scipy.stats import norm, gumbel_l
import pandas as pd
import csv
from copy import deepcopy

# Cell with jit-ted functions that cannot be called from within a class.

@njit
def c_life(whn, T, R, J):
 
    return ((1 - R**(-1)) / (1 - R**(-J))) * T + whn

@njit
def y_life(whn, T, R, J):

    return T + whn * ((1 - R**(-J)) / (1 - R**(-1)))

@njit
def foc_c_n(n, wh, T, R, J, ψ, ε, σ):
    cmin = 1e-5
    c = c_life(wh * n, T, R, J)
    c = np.where(c>cmin, c, cmin)
    foc = wh * du_dc(c, σ) + du_dn(1.0 - n, ψ, ε)
    return foc

@njit
def solve_c_n(wh, T, R, J, ψ, ε, σ):
    Err, I = 1e-9, 10
    n_lo = np.zeros(len(wh)) 
    n_hi = np.ones(len(wh)) 
    foc_max, i = Err + 1, 0  
    
    # Iterate over guesses of n until foc is satisfied
    while foc_max>Err and i<I:
        n = (n_lo + n_hi) / 2
        
        foc = foc_c_n(n, wh, T, R, J, ψ, ε, σ)
        n_lo = np.where(foc<-Err, n_lo, n)
        n_hi = np.where(foc>Err, n_hi, n)
        foc_max = np.amax(np.abs(foc))
        i += 1   
    c = c_life(wh * n, T, R, J)
    
    return c, n

@njit
def util(c, leis, σ, ψ, ε):
    uc = (c**(1 - σ) - 1) / (1 - σ)
    un = (ψ / (1 - ε)) * leis**(1-ε)
    return uc + un

@njit
def du_dc(c, σ):
    """
    Derivative of utility function wrt consumption.
    """
    return c**(-σ)

@njit
def du_dn(leis, ψ, ε):
    """
    Derivative of utility function wrt labor.
    """
    return - ψ * leis**(-ε)

@njit
def γ(h, π1, π2):
    h_bounded = np.maximum(np.minimum(h, 800), 200)
    denom = 1 + np.exp(-(π1 + π2 * h_bounded))
    return 1 / denom

@njit
def v_life(leis, whn, T, J, σ, ψ, ε, β, R):
    y = y_life(whn, T, R, J)
    c = c_life(whn, T, R, J)
    u = util(c, leis, σ, ψ, ε)
    return u * (1 - β**J) / (1 - β)

@njit
def vc_life(w_g, w_ng, n_a_g, n_a_ng, T, h, π1, π2, σ, ψ, ε, β, R, J):
    # Compute value function from attending college prior to realization of grad vs. no grad
    y_a_g = y_life(w_g * h * n_a_g, T, R, J)
    y_a_ng = y_life(w_ng * h * n_a_ng, T, R, J)
    
    c_a_g = c_life(w_g * h * n_a_g, T, R, J)
    c_a_ng = c_life(w_g * h * n_a_g, T, R, J)
    
    v_g = v_life(1.0 - n_a_g, w_g * h * n_a_g, T, J, σ, ψ, ε, β, R)
    v_ng = v_life(1.0 - n_a_ng, w_ng * h * n_a_ng, T, J, σ, ψ, ε, β, R)
    
    return γ(h, π1, π2) * v_g + (1 - γ(h, π1, π2)) * v_ng

# Cell with jit-ted functions that cannot be called from within a class.
@njit
def m_given_i(τi, τm, i, wH, n, θm, ρ):
    # Derive elasticity of substitution
    ε_mi = 1 / (1 - ρ)    
    
    # Derive optimal market investment given time investment
    return  n * i * ( ((wH - τi) / (1 - τm)) * (θm / (1 - θm)) )**ε_mi

@njit
def hc(h, ni, m, a, H, θm, ρ, ϕ, ζ):
    return 300.0 + a * ((H / 500)**ζ) * (θm * m**ρ + (1 - θm) * ni**ρ)**(ϕ/ ρ)

def cov(x, y, w):
    """Weighted Covariance"""
    return np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)

def corr(x, y, w):
    """Weighted Correlation"""
    return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

def sd_weighted(v, λsp, λ):
    mn_PS, vr_PS = np.zeros([2, 2]), np.zeros([2, 2])
    for nP in range(2):
        for nS in range(2):
            v_PS, λ_PS = v[nP, nS, :, :], λ[nP, nS, :, :]
            mn_PS = np.average(v_PS, weights=λ_PS)
            vr_PS[nP, nS] = np.average((v_PS - mn_PS)**2, weights=λ_PS)
    mn = np.average(v, weights=λ)
    vr = np.average((v - mn)**2, weights=λ)
    return vr**0.5, vr_PS**0.5

@njit
def gen_popwgts(λ, λsp, λH, λa, NP, NS, NH, Na):
    for nP in range(NP):
        for S in range(NS):
            for nH in range(NH):
                for na in range(Na):
                    λ[nP, S, nH, na] = λsp[nP, S] * λH[nH] * λa[nP, S, na]
    return λ / np.sum(λ)

@njit
def solve_n(w, p, inv, coh, σ, ψ, ε, minc):
    Err, I = 1e-9, 10
    n_lo = 0.0 
    n_hi = p - inv 
    foc, i = Err + 1, 0  
    
    # Iterate over guesses of n until foc is satisfied
    while np.abs(foc)>Err and i<I:
        n = (n_lo + n_hi) / 2
        
        # Compute marginal cost of working
        focR = du_dn(p - inv - n, ψ, ε)        
        
        # Compute marginal benefit of working
        inc = w * n + coh
        c = max(minc, inc)
        focL = w * du_dc(c, σ)  
        
        # Evaluate foc and update bounds on savings policy
        foc = focL + focR             
        n_lo = n_lo * (foc<-Err) + n * (foc>=-Err)
        n_hi = n * (foc<=Err) + n_hi * (foc>Err) 
        i += 1
    
    if np.abs(foc)>10*Err:
        'solve_n: ERROR! FOC violated. n=', np.round(n, 3), ', c=', np.round(c, 3), ', foc=', np.round(foc, 5)
    
    return c, n

@njit
def solve_kk(w, p, inv, m, gridk_interp, gridcc, R, β, σ, ψ, ε, minc):
    Err, I = 1e-9, 50
    kk_lo = 0.0 
    kk_hi = gridk_interp[0][1]
    foc, i = Err + 1, 0
    
    # Check if individual is borrowing constrained (if focL<focR at k = kk_lo)
    kk = kk_lo
    c, n = solve_n(w, p, inv, -(m + kk), σ, ψ, ε, minc)
    focR = du_dc(c, σ)
    cc = eval_linear(gridk_interp, gridcc, np.array([kk]))
    focL = β * R * du_dc(cc, σ)
    foc = focL - focR
    if foc < Err:
        foc = 0
    
    # Iterate over guesses of kk until foc is satisfied
    while np.abs(foc)>Err and i<I:
        kk = (kk_lo + kk_hi) / 2
        
        # Compute marginal cost of saving
        c, n = solve_n(w, p, inv, -(m + kk), σ, ψ, ε, minc)
        focR = du_dc(c, σ)

        # Compute marginal benefit of saving
        cc = eval_linear(gridk_interp, gridcc, np.array([kk]))
        focL = β * R * du_dc(cc, σ)
        
        # Evaluate foc and update bounds on savings policy
        foc = focL - focR             
        kk_lo = kk_lo * (foc<-Err) + kk * (foc>=-Err)
        kk_hi = kk * (foc<=Err) + kk_hi * (foc>Err) 
        i += 1
    
    if np.abs(foc)>10*Err:
        'solve_kk: ERROR! FOC violated. kk=', np.round(kk, 3), ', c=', np.round(c, 3), ', foc=', np.round(foc, 5)
    
    return c, n, kk

# Cell with jit-ted functions that cannot be called from within a class.
@jit
def solve_policy_p_1(τi, τm, T, Tm, P, W, H, n, h, a, θm, ρ, ϕ, ζ, α, σ, ψ, ε, β, R, gridvv, gridVV, gridCC, gridi, gridK_interp, gridh_interp, minc):
    
    # Set up value grid corresponding to gridi
    gridV, gridN, gridKK = np.zeros_like(gridi), np.zeros_like(gridi), np.zeros_like(gridi)
        
    # Loop over investment grid 
    for ni, i in enumerate(gridi):
        
        # Compute child's expected cont'n value
        m = max(Tm, m_given_i(τi, τm, i, W * H, n, θm, ρ))
        hh = hc(h, i * n, m, a, H, θm, ρ, ϕ, ζ)
        vv = eval_linear(gridh_interp, gridvv, np.array([hh]))
        
        # Compute parent's utilities
        C, N, KK = solve_kk(W * H * n, P, i, m - T - Tm - τm * m - τi * n * i, gridK_interp, gridCC, R, β, σ, ψ, ε, minc)
        VV = eval_linear(gridK_interp, gridVV, np.array([KK]))
        U = util(C, P - (N + i), σ, ψ, ε)
        
        # Store outcomes
        gridV[ni] = U + β * VV + α * vv
        gridN[ni] = N
        gridKK[ni] = KK
        
    # Choose optimal investment level
    ni = np.argmax(gridV)
    i = gridi[ni]
    m = max(Tm, m_given_i(τi, τm, i, W * H, n, θm, ρ))
    hh = hc(h, i * n, m, a, H, θm, ρ, ϕ, ζ) 
    KK = gridKK[ni]
    N = gridN[ni]
    C = W * H * n * N - m + T + Tm + τi * n * i + τm * m - KK
    
    return [C, N, KK, i, m, hh]

class BlandinHerrington:
    """
    This class computes the outcome of the model in Blandin, Herrington.
    College graduates earn a college premium.
    Children choose whether to attend college, which is risky and costly.
    Probability of graduation conditional on attendance depends on the child's human capital.
    Parents choose how much to invest in their child's human capital.

    INPUTS:
    gridwgt:   population weight grid
    minh:      for human capital grid
    maxh:      for human capital grid
    π_1, π2:   parameters governing the graduation probability technology
    τ_m_ps:    market cost of college
    τ_n:       time (opportunity) cost of college
    w_ng:      wage rate of non-college-graduates
    ω_g:       wage premium of college graduates
    nbar:      endowment of labor each period
    per_length:length of model period
    R_yr:      annual interest rate
    β_yr:      annual  intertemporal discount rate
    σ:         CRRA parameter
    loc_ξ:     location parameter for college preference shock
    scale_ξ:   scale parameter for college preference shock
    σ_H:       sd of adult human capital H
    NH:        number of gridpoints for adult human capital H
    NsdH:      number of σ_H for gridH to span
    
    FUNCTIONS:
    solve_p_2(self):               returns life-cycle savings, consumption, and value functions for empty-nester parents
    solve_c_2(self):               returns life-cycle attendance, completion, earnings, savings, consumption, and value functions for adult children
    solve_p_1(self):               returns investment, savings, consumption decisions for parents with children
    output_results(self):          outputs key statistics of model solutions
    output_results_params(self):   outputs parameter values used
    output_figs(self):             outputs figures from model solutions
    """
    
    def __init__(self,
                 λsp,
                 h0=200,
                 minh=200,
                 maxh=800,
                 Nh=301, 
                 θm=0.333,
                 ρ=0.6, 
                 ϕ=0.50,
                 ζ=0.275, 
                 μ_a_ps=np.array([150, 150, 150, 150]).reshape(2,2),
                 σ_a=0.1,
                 Na=21,
                 π1=-4.589,
                 π2=0.010,
                 τ_m_ps=4*np.array([4_558, 8_140, 6_916, 10_498]).reshape(2,2),
                 τ_n=0.27, 
                 w_ng=0.033,
                 ω_g=0.3,
                 nbar=5_200,
                 per_length=18,
                 R_yr=1.04,
                 β_yr=1/1.04,
                 σ=2.00001,
                 ψ=1.0,
                 ε=1.01,
                 α=0.2,
                 scale_ξ=1,
                 loc_ξ_0=2,
                 μ_H=np.log(500), 
                 σ_H=1e-5,
                 Nk=201,
                 NH=1,
                 NsdH=1,
                 maxi=0.25,
                 δi=0.01/3,
                 π1ps=-5.543,
                 π2ps=0.012,       
                 Tm_yr=0.0,
                 τi=0.0,
                 τm=0.0,
                 path='./results/temp/'):
        
        # Store parameter values that are inputs to the class 
        self.λsp = λsp
        self.h0, self.minh, self.maxh, self.Nh = h0, minh, maxh, Nh
        self.θm, self.ρ, self.ϕ, self.ζ = θm, ρ, ϕ, ζ
        self.μ_a_ps, self.σ_a, self.Na = μ_a_ps, σ_a, Na
        self.π1, self.π2, self.τ_m_ps, self.τ_n = π1, π2, τ_m_ps, τ_n
        self.w_ng, self.ω_g, self.nbar, self.per_length, self.R_yr = w_ng, ω_g, nbar, per_length, R_yr
        self.β_yr, self.σ, self.ψ, self.ε, self.α = β_yr, σ-1, ψ, ε, α
        self.scale_ξ, self.loc_ξ_0, self.μ_H, self.σ_H = scale_ξ, loc_ξ_0, μ_H, σ_H
        self.NP, self.NS, self.Nk, self.NH, self.NsdH = 2, 2, Nk, NH, NsdH
        self.maxi, self.δi = maxi, δi
        self.Tm_yr = Tm_yr
        self.path = path
        
        # Store other model parameter values
        self.Nsda = 3
        self.R, self.β = self.R_yr**self.per_length, self.β_yr**self.per_length
        self.Jc, self.Jp = np.round((70 - 18) / self.per_length), np.round((70 - 36) / self.per_length)
        self.w_g = self.w_ng * (1 + self.ω_g)
        #self.n = self.nbar * per_length
        self.pv_prd = ((1 - self.R_yr**(-self.per_length)) / (1 - self.R_yr**(-1)))
        self.n = self.nbar * self.pv_prd
        self.loc_ξ = self.loc_ξ_0 - self.scale_ξ * 0.557
        self.Ni = 1 + int(self.maxi / self.δi)
        self.π1ps, self.π2ps = π1ps, π2ps
        self.π1_ps = self.π1ps * np.ones(4).reshape(2,2)
        self.π2_ps = self.π2ps * np.ones(4).reshape(2,2)
        self.Tm = self.Tm_yr * self.pv_prd
        self.τm, self.τi = τm, τi
        self.Tτ = 0.0
        self.T_yr = 0.0
        self.T = self.T_yr * self.pv_prd
        
        
        # Store other computational parameters
        self.minc = 1e-6
        
        # Store basic 1d grids        
        self.gridh = np.linspace(self.minh, self.maxh, self.Nh)
        
        self.gridlH, self.λH = self.Tauchen_1d(self.μ_H, self.σ_H, self.NH, self.NsdH)
        self.gridH = np.exp(self.gridlH)
        
        self.mink, self.maxk = -0.3 * self.w_g * self.maxh * self.n, 0.3 * self.w_g * self.maxh * self.n
        self.gridk = np.linspace(self.mink, self.maxk, self.Nk)
        
        self.gridi = np.linspace(0, self.maxi, self.Ni)
        
        self.gridsla = np.zeros([self.NP, self.NS, self.Na])
        self.λa = np.zeros([self.NP, self.NS, self.Na])
        for nP in range(self.NP):
            for S in range(self.NS):
                μ_a = self.μ_a_ps[nP, S]
                self.gridsla[nP, S, :], self.λa[nP, S, :] = self.Tauchen_1d(np.log(μ_a), self.σ_a, self.Na, self.Nsda)
        self.gridsa = np.exp(self.gridsla)
        
        # Store 4d grid of population weights
        self.λ = np.zeros([self.NP, self.NS, self.NH, self.Na])
        self.λ = gen_popwgts(self.λ, self.λsp, self.λH, self.λa, self.NP, self.NS, self.NH, self.Na)
        
        # Store 3d grid of parent state variables
        self.Shape = (self.NP, self.NS, self.NH, self.Nk)
        self.K, self.H, self.P, self.W = np.zeros(self.Shape), np.zeros(self.Shape), np.zeros(self.Shape), np.zeros(self.Shape)
        for nP in range(self.NP):
            self.P[nP, :, :, :] = 1 + nP
        self.W[:, 0, :, :] = self.w_ng
        self.W[:, 1, :, :] = self.w_g  
        for nH in range(self.NH):
            self.H[:, :, nH, :] = self.gridH[nH]        
        for nk in range(self.Nk):
            self.K[:, :, :, nk] = self.gridk[nk]
            
        # Generate grids for interpolation during model solution
        self.gridK_interp = UCGrid((self.mink, self.maxk, self.Nk))
        self.gridh_interp = UCGrid((self.minh, self.maxh, self.Nh))

    def Tauchen_1d(self, μ, σ, S, Nsd):
        """
        This function uses a modified Tauchen (1986) method
        to approximate a Normal distribution.
        
        Distribution: s ~ N(μ, σ**2).
        
        INPUTS:  -μ: mean
                 -σ: SD
                 -S: number of gridpoints
                 -Nsd: number of SD from mean for grid to span
        
        OUTPUTS: -λ, weights on each gridpoint
        """
        
        # Compute grid over state s and the half-distance between gridpoints, δ
        grid = np.linspace(μ - Nsd * σ, μ + Nsd * σ, S)
        δ = (grid[-1] - grid[0]) / (S - 1) / 2
        
        # compute cumulative probabilities of grids
        λcum = np.ones(S) 
        for s in range(S-1):
            λcum[s] = norm.cdf(grid[s] + δ, loc = μ, scale = σ)
        
        # compute probabilities of grids
        λ = λcum
        λ[1:] = λcum[1:] - λcum[:-1]

        # Check λ
        if np.abs(np.sum(λ) - 1)>1e-9:
            print('BlandinHerrington.Tauchen_1d: Distribution grid does not sum to one!')            
            
        return [grid, λ]
        
    def solve_p_2(self):
        # Unpack parameters for clarity
        W, H, K, R, P, n, Jp, σ, ψ, ε, β = self.W, self.H, self.K, self.R, self.P, self.n, self.Jp, self.σ, self.ψ, self.ε, self.β
        NP, NS, NH, Nk = self.NP, self.NS, self.NH, self.Nk
        
        # Solve lifetime consumption, labor, income given state (P, S, K)
        Cp, Np = np.zeros([NP, NS, NH, Nk]), np.zeros([NP, NS, NH, Nk])
        for nP in range(NP):
            for S in range (NS):
                for nH in range(NH):
                    Cp[nP, S, nH, :], Np[nP, S, nH, :] = solve_c_n(W[nP, S, nH, :] * H[nP, S, nH, :] * P[nP, S, nH, :] * n, R * K[nP, S, nH, :], R, Jp, ψ, ε, σ)
        Np = P * Np
        Yp = W * H * P * n * Np

        # Compute value functions given lifetime incomes
        Vp = v_life(P - Np, W * H * P * n * Np, R * K, Jp, σ, ψ, ε, β, R)
             
        return Vp, Yp, Cp, Np


    def solve_c_2(self, h, T): 
        # Unpack variables for simpler notation
        w_ng, w_g, τ_m_ps, τ_n, π1_ps, π2_ps, loc_ξ, scale_ξ, σ, ψ, ε, β, R, Jc, nbar, n = self.w_ng, self.w_g, self.τ_m_ps, self.τ_n, self.π1_ps, self.π2_ps, self.loc_ξ, self.scale_ξ, self.σ, self.ψ, self.ε, self.β, self.R, self.Jc, self.nbar, self.n
        NP, NS, Nh = self.NP, self.NS, self.Nh
        
        # Copute lifetime labor and income given na, a_dropout, a_graduate
        coh_na = T
        c_na, n_na = solve_c_n(w_ng * h * n, coh_na, R, Jc, ψ, ε, σ)
        coh_a, c_a_ng, c_a_g, n_a_ng, n_a_g = np.zeros([NP, NS, Nh]), np.zeros([NP, NS, Nh]), np.zeros([NP, NS, Nh]), np.zeros([NP, NS, Nh]), np.zeros([NP, NS, Nh])
        for nP in range(NP):
            for S in range (NS):
                coh_a[nP, S, :] = T - τ_m_ps[nP, S] - w_ng * h * τ_n * nbar * 4
                c_a_ng[nP, S, :], n_a_ng[nP, S, :] = solve_c_n(w_ng * h * n, coh_a[nP, S, :], R, Jc, ψ, ε, σ)
                c_a_g[nP, S, :], n_a_g[nP, S, :] = solve_c_n(w_g * h * n, coh_a[nP, S, :], R, Jc, ψ, ε, σ)
        
        y_na = y_life(w_ng * h * n * n_na, coh_na, R, Jc)
        y_a_ng = y_life(w_ng * h * n * n_a_ng, coh_a, R, Jc)
        y_a_g = y_life(w_g * h * n * n_a_g, coh_a, R, Jc)

        # Compute value functions given lifetime choices
        v_a, v_na = np.zeros([NP, NS, Nh]), np.zeros([NP, NS, Nh])
        v_na_temp = v_life(1.0 - n_na, w_ng * h * n * n_na, coh_na, Jc, σ, ψ, ε, β, R)
        for nP in range(NP):
            for S in range (NS):
                π1, π2 = π1_ps[nP, S], π2_ps[nP, S]
                v_a[nP, S, :] = vc_life(w_g * n, w_ng * n, n_a_g[nP, S, :], n_a_ng[nP, S, :], coh_a[nP, S, :], h, π1, π2, σ, ψ, ε, β, R, Jc)
                v_na[nP, S, :] = v_na_temp
        
        # Compute expected model outcomes (taking expectation over ξ)
        prob_a_ps = gumbel_l.cdf(v_a - v_na, loc=loc_ξ, scale=scale_ξ)
        
        # Compute expected value function given hc
        v = prob_a_ps * v_a + (1 - prob_a_ps) * v_na

        return prob_a_ps, v, v_na, v_a, y_na, y_a_ng, y_a_g, c_na, c_a_ng, c_a_g, n_na, n_a_ng, n_a_g
    

    def solve_p_1(self, prob_a_ps, vv, VV, CC, n_na, n_a_ng, n_a_g):
        # Unpack parameters for clarity
        NP, NS, NH, Na = self.NP, self.NS, self.NH, self.Na
        h0, gridsa, θm, ρ, ϕ, ζ, π1_ps, π2_ps = self.h0, self.gridsa, self.θm, self.ρ, self.ϕ, self.ζ, self.π1_ps, self.π2_ps
        n, w_ng, w_g = self.n, self.w_ng, self.w_g
        α, σ, ψ, ε, β, R, minc = self.α, self.σ, self.ψ, self.ε, self.β, self.R, self.minc
        gridK_interp, gridh_interp, gridH, gridi = self.gridK_interp, self.gridh_interp, self.gridH, self.gridi
        τi, τm, Tm, T = self.τi, self.τm, self.Tm, self.T
        
        # Initialize policy shells
        mC, mN, mKK, mi, mm, mhh = np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na])
        
        # Loop over parent state space
        for nP in range(NP):
            P = nP + 1
            for S in range(NS):
                W = w_ng * (S==0) + w_g * (S==1)
                grida = gridsa[nP, S, :]
                for nH in range(NH):
                    H = gridH[nH]
                    
                    for na in range(Na):
                        a = grida[na]
                        # Solve for optimal policy
                        [C, N, KK, i, m, hh] = solve_policy_p_1(τi, τm, T, Tm, P, W, H, n, h0, a, θm, ρ, ϕ, ζ, α, σ, ψ, ε, β, R, vv[nP, S, :], VV[nP, S, nH, :], CC[nP, S, nH, :], gridi, gridK_interp, gridh_interp, minc)
                        
                        # Adjust nominal figures into annual values
                        C, KK, m = C / self.pv_prd, KK / self.pv_prd, m / self.pv_prd
                        # Store results
                        mC[nP, S, nH, na], mN[nP, S, nH, na], mKK[nP, S, nH, na], mi[nP, S, nH, na], mm[nP, S, nH, na], mhh[nP, S, nH, na] = [C, N, KK, i, m, hh]

        # Compute attendance probabilities
        mattend = np.zeros([NP, NS, NH, Na])
        for nP in range(NP):
            for S in range(NS):
                prob_a = prob_a_ps[nP, S, :]        
                pts_hh = np.array([mhh[nP, S, :, :].reshape(NH * Na)]).T
                mattend_temp = 1e-5 + eval_linear(gridh_interp, prob_a, pts_hh)
                mattend_temp = mattend_temp.reshape(NH, Na)
                mattend[nP, S, :, :] = mattend_temp.reshape(NH, Na)
        
        # Compute graduation probabilities (unconditional on attendance)
        mgrad = np.zeros([NP, NS, NH, Na])
        for nP in range(NP):
            for S in range(NS):
                π1, π2 = π1_ps[nP, S], π2_ps[nP, S]
                mgrad[nP, S, :, :] = γ(mhh[nP, S, :, :], π1, π2) * mattend[nP, S, :, :]
                
        
        # Compute mean hours (unconditional on attendance)
        # and hours given non-attendance, hours given attend/non-graduate, attend/graduate
        mn, mn_na, mn_ng, mn_g = np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na]), np.zeros([NP, NS, NH, Na])
        for nP in range(NP):
            for S in range(NS):
                pts_hh = np.array([mhh[nP, S, :, :].reshape(NH * Na)]).T
                nna, na_ng, na_g = eval_linear(gridh_interp, n_na, pts_hh), eval_linear(gridh_interp, n_a_ng[nP, S, :], pts_hh), eval_linear(gridh_interp, n_a_g[nP, S, :], pts_hh)
                nna, na_ng, na_g = nna.reshape(NH, Na), na_ng.reshape(NH, Na), na_g.reshape(NH, Na)
                mn_na[nP, S, :, :], mn_ng[nP, S, :, :], mn_g[nP, S, :, :] = nna, na_ng, na_g
                pna, pa_ng, pa_g = 1 - mattend[nP, S, :, :], mattend[nP, S, :, :] - mgrad[nP, S, :, :], mgrad[nP, S, :, :]
                mn[nP, S, :, :] = pna * nna + pa_ng * na_ng + pa_g * na_g 

        return [mC, mN, mKK, mi, mm, mhh, mattend, mgrad, mn, mn_na, mn_ng, mn_g]


    def gen_stats_earn(self, mN, mi, mhh, mattend, mgrad, y2_na, y2_a_ng, y2_a_g):
        # Unpack parameters
        NP, NS, NH, Na = self.NP, self.NS, self.NH, self.Na
        π1_ps, π2_ps, w_ng, w_g, n, gridH = self.π1_ps, self.π2_ps, self.w_ng, self.w_g, self.n, self.gridH
        
        # Set up variable shells
        mE  = np.zeros([NP, NS, NH, Na])
        meph_na, meph_ang, meph_g, meph, me, me_net = np.zeros_like(mE), np.zeros_like(mE), np.zeros_like(mE), np.zeros_like(mE), np.zeros_like(mE), np.zeros_like(mE)

        # Loop over parent state space
        for nP in range(NP):
            P = nP + 1
            for S in range(NS):
                W = w_ng * (S==0) + w_g * (S==1)
                π1, π2 = π1_ps[nP, S], π2_ps[nP, S]
                for nH in range(NH):
                    H = n * gridH[nH]
                    for na in range(Na):
                        # Parent earnings
                        mE[nP, S, nH, na] = W * H * mN[nP, S, nH, na] # * P
                        
                        # Child earnings
                        hh = mhh[nP, S, nH, na]
                        proba, probg_a = mattend[nP, S, nH, na], γ(hh, π1, π2)
                        pts_hh = np.array([hh])
                        y_na = eval_linear(self.gridh_interp, y2_na, pts_hh)
                        y_ng = eval_linear(self.gridh_interp, y2_a_ng[nP, S, :], pts_hh)
                        y_g = eval_linear(self.gridh_interp, y2_a_g[nP, S, :], pts_hh)
                        meph_na[nP, S, nH, na] = w_ng * hh
                        meph_ang[nP, S, nH, na] = w_ng * hh
                        meph_g[nP, S, nH, na] = w_g * hh
                        meph[nP, S, nH, na] = (1 - proba) * meph_na[nP, S, nH, na] + (proba - probg_a) * meph_ang[nP, S, nH, na] + probg_a * meph_g[nP, S, nH, na]

                        # child earnings, gross and net of tuition
                        me[nP, S, nH, na] = (1 - proba) * y_na + (proba - probg_a) * (y_ng + self.τ_m_ps[nP, S]) + probg_a * (y_g + self.τ_m_ps[nP, S])
                        me_net[nP, S, nH, na] = (1 - proba) * y_na + (proba - probg_a) * y_ng + probg_a * y_g

        return mE, meph_na, meph_ang, meph_g, meph, me, me_net
    
    
    def reg_gradprob(self, mE, mgrad):
        """
        regress college grad probability on log parent earnings and family type
        """
        # infer population sizes
        NP, NS, NH, Na = np.shape(mE)
        N, Nf = np.prod(np.shape(mE)), NP * NS

        # construct matrix of family types 
        mftype = np.zeros_like(mE)
        for nP in range(NP):
            for nS in range(NS):
                mftype[nP, nS, :, :] = nP * NS + nS
        mftype = mftype.astype(int)

        # construct regression variables (dummies for family type)
        y, x, λ= mgrad.reshape(N), np.log(mE.reshape(N)), self.λ.reshape(N)
        for f in np.arange(Nf):
            x = np.vstack((x, np.where(mftype==f, 1, 0).reshape(N)))            
        x = x.T

        # run regression
        model = sm.WLS(y, x, weights=λ)
        results = model.fit()
        return results    
        

    def gen_stats(self, mN, mi, mm, mhh, mattend, mgrad, mn, y2_na, y2_a_ng, y2_a_g):
        
        ### Compute earnings-related stats
        mE, meph_na, meph_ang, meph_g, meph, me, me_net = self.gen_stats_earn(mN, mi, mhh, mattend, mgrad, y2_na, y2_a_ng, y2_a_g)

        ### Compute means by SP and aggregate means
        # Unconditional means
        d_ps, d_ps_sd, d_agg, d_agg_sd = {}, {}, {}, {}
        variables = [np.log(mE), mN, mi, mm, mhh, mattend, mgrad, meph, mn, me, me_net, np.log(me), np.log(me_net)]
        labels = ['lE', 'N', 'i', 'm', 'hh', 'sh_a', 'sh_g', 'eph', 'n', 'e', 'e_net', 'le', 'le_net']
        for lbl, v in zip(labels, variables):
            if lbl=='i':
                v[1, :, :, :] = v[1, :, :, :] / 2
            d_ps[lbl] = np.average(v, axis=(2, 3), weights=self.λ)
            d_agg[lbl] = np.average(v, weights=self.λ)
            d_agg_sd["{}".format(lbl)], d_ps_sd["{}".format(lbl)] = sd_weighted(v, self.λsp, self.λ)
        
        # Conditional means
        variables = [mhh, mhh, mhh, np.log(meph_na), np.log(meph_ang), np.log(meph_g), np.log(me), np.log(me_net)]
        labels = ['hh_na', 'hh_a', 'hh_g', 'leph_na', 'leph_a', 'leph_g', 'le', 'le_net']
        probs = [1 - mattend, mattend, mgrad, 1 - mattend, mattend, mgrad]
        for lbl, v, p in zip(labels, variables, probs):
            d_ps["{}".format(lbl)] = np.average(v, axis=(2, 3), weights=self.λ * p)
            d_agg["{}".format(lbl)] = np.average(v, weights=self.λ * p)
            d_agg_sd["{}".format(lbl)], d_ps_sd["{}".format(lbl)] = sd_weighted(v, self.λsp, self.λ * p)
        
        # regress college grad probability on log parent earnings and family type
        reg_gradprob_results = self.reg_gradprob(mE, mgrad)
        
        return d_ps, d_agg, d_agg_sd, reg_gradprob_results

    
    def maketab5(self, reg):
        # output results of college completion regression
        est = reg.params
        est[1:] = est[1:] - est[3]
        est = np.hstack((est[:3], est[4]))
        fpath = '../output/tables/table5.csv'
        with open(fpath, 'w', newline='') as f:
            pen = csv.writer(f)
            pen.writerow([' ', 'Data coefficient', 'Model coefficient', 'Model / Data'])
            pen.writerow(['1L', -0.076, np.round(est[1], 3), np.round(est[1] / -0.076, 2)])
            pen.writerow([' ', 0.018])
            pen.writerow(['1H', 0.107,  np.round(est[2], 3), np.round(est[2] / 0.107, 2)])
            pen.writerow([' ', 0.035])
            pen.writerow(['2H', 0.318,  np.round(est[3], 3), np.round(est[3] / 0.318, 2)])
            pen.writerow([' ', 0.020])
            pen.writerow(['Parent earnings', 0.092,  np.round(est[0], 3), np.round(est[0] / 0.092, 2)])
            pen.writerow([' ', 0.010])
    
    
    def output_results(self, d_ps, d_agg, d_agg_sd, prob_a_ps, reg, path_data, output=False):
        #self.output_results_params()
        self.output_results_stats(d_ps, d_agg, d_agg_sd, reg)
        self.output_results_statsvsdata(d_ps, d_agg, path_data)

    def output_results_params(self):
        #List of parameter titles
        titles = ['taui', 'taum', 'T_yr', 'T', 'Tm_yr', 'Tm', 'Ttuit', 'minh', 'maxh', 'Nh', 'NH', 'mu_H', 'sigma_H', 'mu_a_ps', 'sigma_a', 'Na', 'psi', 'elast', 'alpha', 'theta_m', 'phi', 'rho', 'pi1_ps', 'pi2_ps', 'tau_m_ps', 'tau_n', 'omega_g', 'w_ng', 'w_g', 'nbar', 'R', 'Jc', 'Jp', 'beta', 'sigma', 'scale_xi', 'loc_xi_0', 'zeta']
        values = [self.τi, self.τm, self.T_yr, self.T, self.Tm_yr, self.Tm, self.Tτ, self.minh, self.maxh, self.Nh, self.NH, self.μ_H, self.σ_H, self.μ_a_ps, self.σ_a, self.Na, self.ψ, self.ε, self.α, self.θm, self.ϕ, self.ρ, self.π1, self.π2, self.τ_m_ps, self.τ_n, self.ω_g, self.w_ng, self.w_g, self.nbar, self.R, self.Jc, self.Jp, self.β, self.σ, self.scale_ξ, self.loc_ξ_0, self.ζ]

        # Initialize csv I will write parameters to
        fpath = self.path + 'params.csv'
        with open(fpath, 'w', newline='') as f:
            pen = csv.writer(f)
            pen.writerow(titles)
            pen.writerow(values)


    def output_results_stats(self, d_ps, d_agg, d_agg_sd, reg):
        fullpath = self.path + 'stats.csv'
        est, se = list(reg.params), list(reg.bse)
        est[1:] = est[1:] - est[3]
        with open(fullpath, 'w', newline='') as f:
            pen = csv.writer(f)
            pen.writerow(['variable', 'Agg, Mn', 'Agg, SD', '1L', '1H', '2L', '2H']) 
            weights = ['w_ps', np.sum(self.λsp), '']
            weights.extend(self.λsp.reshape(4).tolist())
            pen.writerow(weights)
            for lbl in d_agg.keys():
                if lbl!='i':
                    statlist = [lbl, d_agg[lbl], d_agg_sd[lbl]]
                    statlist.extend(d_ps[lbl].reshape(4).tolist())
                else:
                    statlist = [lbl, 100 * d_agg[lbl], 100 * d_agg_sd[lbl]]
                    statlist.extend((100 * d_ps[lbl]).reshape(4).tolist())
                pen.writerow(statlist)
             
                
    def output_results_statsvsdata(self, d_ps, d_agg, path_data):
        # Set path for output
        fullpath = self.path + 'statsvsdata.csv'
        
        # Import data moments
        df = pd.read_csv(path_data)
        df = df.transpose()
        df.columns = df.iloc[0]
        labels = list(df)
        list(df['i'])
        
        # Write model and data moments to output
        with open(fullpath, 'w', newline='') as f:
            pen = csv.writer(f)
            pen.writerow(['DATA'])
            pen.writerow(['Variable', 'Agg, Mn', '1L', '1H', '2L', '2H']) 
            for lbl in labels:
                pen.writerow(list(df[lbl]))
            labels.remove('w_ps')
            pen.writerow(['MODEL'])
            pen.writerow(['Variable', 'Agg, Mn', '1L', '1H', '2L', '2H']) 
            weights = ['w_ps', np.sum(self.λsp)]
            weights.extend(self.λsp.reshape(4).tolist())
            pen.writerow(weights)
            for lbl in labels:
                statlist = [lbl, d_agg[lbl]]
                statlist.extend(d_ps[lbl].reshape(4).tolist())
                pen.writerow(statlist)
                             
                             
    def output_results_figs(self, prob_a_ps):
        # Unpack parameters
        gridh, π1_ps, π2_ps = self.gridh, self.π1_ps, self.π2_ps
        
        ### Loop over (P, S) states
        for nP in range(self.NP):
            for S in range(self.NS):
                π1, π2 = π1_ps[nP, S], π2_ps[nP, S]
                prob_a = prob_a_ps[nP, S, :]
                
                # Compute necessary profiles
                prob_g_a = γ(gridh, π1, π2)
                prob_g = prob_g_a * prob_a

                # Plot attendance probability as a function of hh
                fig, ax = plt.subplots()
                ax.plot(gridh, prob_a, linestyle='dotted', label='Attendance')
                ax.plot(gridh, prob_g_a, linestyle='dashed', label='Graduation, Conditional')
                ax.plot(gridh, prob_g, label='Graduation, Unconditional')
                ax.set_xlabel('Child Human Capital (SAT)')
                ax.set_ylabel('Share of Population')
                ax.legend()
                plt.show()
                fullpath = self.path + 'attainment_profiles_P{}_S{}.pdf'.format(nP, S)
                fig.savefig(fullpath, bbox_inches='tight') 

def run_parametrized_model(bh, path_data, output=False):
    prob_a, v2, v2_na, v2_a, y2_na, y2_a_ng, y2_a_g, c_na, c_a_ng, c_a_g, n_na, n_a_ng, n_a_g = bh.solve_c_2(h=bh.gridh, T=0)
    V2, Y2, C2, N2 = bh.solve_p_2()
    [mC, mN, mKK, mi, mm, mhh, mattend, mgrad, mn, mn_na, mn_ng, mn_g] = bh.solve_p_1(prob_a, v2, V2, C2, n_na, n_a_ng, n_a_g)
    d_ps, d_agg, d_agg_sd, reg_gradprob_results = bh.gen_stats(mN, mi, mm, mhh, mattend, mgrad, mn, y2_na, y2_a_ng, y2_a_g)
    if output==True:
        bh.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return d_ps, d_agg, d_agg_sd, prob_a, mhh, reg_gradprob_results

def run_counterfactual(bh_in, path_data, invonly=False, prob_a_fixed=0, τ_m_ps=4*np.array([3_345, 5_657, 2_534, 4_846]).reshape(2,2)):
    bh = deepcopy(bh_in)
    bh.τ_m_ps = τ_m_ps
    prob_a, v2, v2_na, v2_a, y2_na, y2_a_ng, y2_a_g, c_na, c_a_ng, c_a_g, n_na, n_a_ng, n_a_g = bh.solve_c_2(h=bh.gridh, T=0)
    if invonly==True:
        v2 = prob_a * v2_a + (1 - prob_a) * v2_na
        prob_a = prob_a_fixed
    V2, Y2, C2, N2 = bh.solve_p_2()
    [mC, mN, mKK, mi, mm, mhh, mattend, mgrad, mn, mn_na, mn_ng, mn_g] = bh.solve_p_1(prob_a, v2, V2, C2, n_na, n_a_ng, n_a_g)
    d_ps, d_agg, d_agg_sd, reg_gradprob_results = bh.gen_stats(mN, mi, mm, mhh, mattend, mgrad, mn, y2_na, y2_a_ng, y2_a_g)
    return d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results

def decompose_ineq(bh_in, ω_g, λsp, path_ctrf, path_data):
    # solve model using input parameters
    bh_out = deepcopy(bh_in)
    prob_a, v2, v2_na, v2_a, y2_na, y2_a_ng, y2_a_g, c_na, c_a_ng, c_a_g, n_na, n_a_ng, n_a_g = bh_out.solve_c_2(h=bh_out.gridh, T=0)
    V2, Y2, C2, N2 = bh_out.solve_p_2()
    [mC, mN, mKK, mi, mm, mhh, mattend, mgrad, mn, mn_na, mn_ng, mn_g] = bh_out.solve_p_1(prob_a, v2, V2, C2, n_na, n_a_ng, n_a_g)
    
    # load in counterfactual parameters
    bh_out.ω_g = ω_g
    bh_out.w_g = bh_out.w_ng * (1 + bh_out.ω_g)
    bh_out.λsp = λsp
    bh_out.λa = np.zeros([bh_out.NP, bh_out.NS, bh_out.Na])
    for nP in range(bh_out.NP):
        for S in range(bh_out.NS):
            μ_a = bh_out.μ_a_ps[nP, S]
            bh_out.gridsla[nP, S, :], bh_out.λa[nP, S, :] = bh_out.Tauchen_1d(np.log(μ_a), bh_out.σ_a, bh_out.Na, bh_out.Nsda)
    bh_out.gridsa = np.exp(bh_out.gridsla)
    bh_out.λ = np.zeros([bh_out.NP, bh_out.NS, bh_out.NH, bh_out.Na])
    bh_out.λ = gen_popwgts(bh_out.λ, bh_out.λsp, bh_out.λH, bh_out.λa, bh_out.NP, bh_out.NS, bh_out.NH, bh_out.Na)
    bh_out.path = path_ctrf
    
    # compute stats with counterfactual parameters
    d_ps, d_agg, d_agg_sd, reg_gradprob_results = bh_out.gen_stats(mN, mi, mm, mhh, mattend, mgrad, mn, y2_na, y2_a_ng, y2_a_g)
    bh_out.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return bh_out

def compare_baseline_ctrf(bh_early, bh_late):
    
    # Read stats from baseline and counterfactual models
    df_1 = pd.read_csv(bh_early.path + 'stats.csv')
    df_2 = pd.read_csv(bh_late.path + 'stats.csv')  
    
    # Compute difference between later and earlier
    df_3 = df_1.copy()
    df_3.iloc[:, 1:] = df_2.iloc[:, 1:] - df_1.iloc[:, 1:]
    
    # Write to new csv the difference btween counterfactual and baseline
    fpath_3 = bh_early.path + 'stats_change.csv'
    df_3.to_csv(fpath_3)
    
def counterfactual_isubsidy(bh, path_data, T=0.0, minτi=0.0, maxτi=12, Nτi=40):
    Err, I = 1, Nτi
    diff, i = Err + 1, 0
    τi_lo, τi_hi = minτi, maxτi

    # Iterate over guesses of τi unτil avg. hh_1L matches avg. hh_2L
    while np.abs(diff)>Err and i<I:
        τi = (τi_lo + τi_hi) / 2
        
        # Solve model given τi
        bh.τi = τi
        d_ps, d_agg, d_agg_sd, prob_a, hh, reg_gradprob_results = run_parametrized_model(bh, path_data)
        
        # Compare average hh_1L to target hh_2L
        T_implied = bh.n * d_ps['i'][0, 0] * τi
        diff = T_implied - T*1.25
        
        # Adjust guess of τi accordingly
        τi_lo = np.where(diff>Err, τi_lo, τi)
        τi_hi = np.where(diff<-Err, τi_hi, τi)
        i += 1
    
    bh.τi_yr = bh.τi / bh.pv_prd
    bh.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return 

def counterfactual_msubsidy(bh, path_data, T=0.0, minτm=0.0, maxτm=1, Nτm=40):
    Err, I = 1, Nτm
    diff, i = Err + 1, 0
    τm_lo, τm_hi = minτm, maxτm

    # Iterate over guesses of τm unτml avg. hh_1L matches avg. hh_2L
    while np.abs(diff)>Err and i<I:
        τm = (τm_lo + τm_hi) / 2
        
        # Solve model given τm
        bh.τm = τm
        d_ps, d_agg, d_agg_sd, prob_a, hh, reg_gradprob_results = run_parametrized_model(bh, path_data)
        
        # Compare average hh_1L to target hh_2L
        T_implied = d_ps['m'][0, 0] * τm
        diff = T_implied - T*1.25     
        
        # Adjust guess of τm accordingly
        τm_lo = np.where(diff>Err, τm_lo, τm)
        τm_hi = np.where(diff<-Err, τm_hi, τm)
        i += 1
    
    bh.τm_yr = bh.τm / bh.pv_prd
    bh.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return 

def counterfactual_tuitionsubsidy(bh, path_data, attend_2L=.605, minTτ=0.0, maxTτ=40_000, NTτ=40):
    Err, I = 1e-3, NTτ
    diff, i = Err + 1, 0
    Tτ_lo, Tτ_hi = minTτ, maxTτ
    τ_0 = bh.τ_m_ps[0, 0]

    # Iterate over guesses of Tτ until avg. attend_1L matches attend_2L
    while np.abs(diff)>Err and i<I:
        Tτ = (Tτ_lo + Tτ_hi) / 2
        
        # Solve model given Tτ
        bh.Tτ = Tτ
        bh.τ_m_ps = (τ_0 - bh.Tτ) * np.ones([2, 2])
        d_ps, d_agg, d_agg_sd, prob_a, hh, reg_gradprob_results = run_parametrized_model(bh, path_data)
        
        # Compare average hh_1L to target hh_2L
        attend_1L = d_ps['sh_a'][0, 0]
        diff = attend_1L - attend_2L
        
        # Adjust guess of Tτ accordingly
        Tτ_lo = np.where(diff>Err, Tτ_lo, Tτ)
        Tτ_hi = np.where(diff<-Err, Tτ_hi, Tτ)
        i += 1
    
    bh.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return Tτ

def counterfactual_mtransfer(bh, path_data, attend_2L=.605, minTm=0.0, maxTm=100_000, NTm=40):
    Err, I = 1e-3, NTm
    diff, i = Err + 1, 0
    Tm_lo, Tm_hi = minTm, maxTm

    # Iterate over guesses of Tm until avg. hh_1L matches avg. hh_2L
    while np.abs(diff)>Err and i<I:
        Tm = (Tm_lo + Tm_hi) / 2
        
        # Solve model given Tm
        bh.Tm = Tm
        d_ps, d_agg, d_agg_sd, prob_a, hh, reg_gradprob_results = run_parametrized_model(bh, path_data)
        
        # Compare average hh_1L to target hh_2L
        attend_1L = d_ps['sh_a'][0, 0]
        diff = attend_1L - attend_2L
        
        # Adjust guess of Tm accordingly
        Tm_lo = np.where(diff>Err, Tm_lo, Tm)
        Tm_hi = np.where(diff<-Err, Tm_hi, Tm)
        i += 1
    
    bh.Tm_yr = bh.Tm / bh.pv_prd
    bh.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return 

def counterfactual_transfer(bh, path_data, T=0.0):

    bh.T = T
    d_ps, d_agg, d_agg_sd, prob_a, hh, reg_gradprob_results = run_parametrized_model(bh, path_data)
    
    bh.T_yr = bh.T / bh.pv_prd
    bh.output_results(d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results, path_data)
    return 

def calibrate(mn_m,
              mn_i,
              mn_prem,
              sd_hh,
              share_a,
              share_a_ps,
              mn_hh,
              mn_hh_ps,
              λsp, 
              σ,
              ε=2.0, 
              ψ=1.0,
              ρ=0.7,
              σ_a=.175,
              ζ=0.275,
              σ_H=1e-5,
              NH=2,
              NsdH=1.0, 
              scale_ξ=1.0,
              maketab=False,
              path='./results/temp/',
              path_data=''):
    
    ### Initialize parameters for calibration
    params = {0: 'theta_m', 1: 'alpha', 2: 'omega_g / omega_ng', 3: 'mu_xi', 4: 'mu_a_g', 5: 'phi', 6: 'mu_a_ng', 7: 'eta', 8: 'psi', 9: 'sigma_a', 10: 'sigma_xi', 11: 'sigma_H'}
    moments = {0: 'mean_m', 1: 'mean_i', 2: 'e_g / e_ng', 3: 'share_a', 4: 'mean_hh_2h', 5: 'mean_hh_2l', 6: 'mean_hh_1l', 7: 'coeff_loge', 8: 'mean_n', 9: 'sd_hh', 10: 'corr(hh, attend)', 11: 'var(log E)'}
    Err = {0: 1, 1: .001, 2: .005, 3: .002, 4: 1.0, 5: 1.0, 6: 1.0}
    I = {0: 7, 1: 5, 2: 10, 3: 10, 4: 15, 5: 1, 6: 15} 
    bnd = {0: [0.03, 0.06], 1: [0.6, 0.8], 2: [0.30, 0.70], 3: [-2.0, 1.0], 4: [0.0, 200.0], 5: [0.26, 0.26], 6: [0.0, 160.0]}

    ### Perform nested calibration
    bnd0 = np.copy(bnd[0])
    err0, i0 = 1 + Err[0], 0
    while np.abs(err0)>Err[0] and i0<I[0]:
        θm = (bnd0[0] + bnd0[1]) / 2
        
        bnd1 = np.copy(bnd[1])
        err1, i1 = 1 + Err[1], 0
        while np.abs(err1)>Err[1] and i1<I[1]:
            α = (bnd1[0] + bnd1[1]) / 2

            bnd2 = np.copy(bnd[2])
            err2, i2 = 1 + Err[2], 0
            while np.abs(err2)>Err[2] and i2<I[2]:
                ω_g = (bnd2[0] + bnd2[1]) / 2

                bnd3 = np.copy(bnd[3])
                err3, i3 = 1 + Err[3], 0
                while np.abs(err3)>Err[3] and i3<I[3]:
                    loc_ξ_0 = (bnd3[0] + bnd3[1]) / 2

                    bnd4 = np.copy(bnd[4])
                    err4, i4 = 1 + Err[4], 0
                    
                    μ_a_g = (bnd4[0] + bnd4[1]) / 2
                    
                    bnd5 = np.copy(bnd[5])
                    err5, i5 = 1 + Err[5], 0
                    while np.max(np.abs(err5))>Err[5] and i5<I[5]:
                        ϕ = (bnd5[0] + bnd5[1]) / 2                    

                        bnd6 = np.copy(bnd[6])
                        err6, i6 = 1 + Err[6], 0
                        while np.max(np.abs(err6))>Err[6] and i6<I[6]:
                            μ_a_ng = (bnd6[0] + bnd6[1]) / 2
                            μ_a_ps = np.array([μ_a_ng, μ_a_g, μ_a_ng, μ_a_g]).reshape(2,2)

                            # Solve model given parameter values
                            bh = BlandinHerrington(λsp=λsp, μ_a_ps=μ_a_ps, loc_ξ_0=loc_ξ_0, ϕ=ϕ, ω_g=ω_g, α=α, ψ=ψ, ε=ε, σ=σ, σ_a=σ_a, ζ=ζ, σ_H=σ_H, NH=NH, NsdH=NsdH, scale_ξ=scale_ξ, θm=θm, ρ=ρ, path=path)
                            d_ps, d_agg, d_agg_sd, prob_a, hh, reg = run_parametrized_model(bh, path_data)

                            # Evaluate target 6 and adjust bnd5
                            err6 = d_ps['hh'][0, 0] - mn_hh_ps[0, 0]
                            err6 = np.nan_to_num(err6)
                            bnd6[0] = np.where(err6>Err[6], bnd6[0], μ_a_ng)
                            bnd6[1] = np.where(err6>=-Err[6], μ_a_ng, bnd6[1])                        
                            i6 += 1 

                        # Evaluate target 5 and adjust bnd5
                        err5 = d_ps['hh'][1, 0] - mn_hh_ps[1, 0]
                        err5 = np.nan_to_num(err5)
                        bnd5[0] = np.where(err5>Err[5], bnd5[0], ϕ)
                        bnd5[1] = np.where(err5>=-Err[5], ϕ, bnd5[1])                        
                        i5 += 1                         
                    
                    while np.abs(err4)>Err[4] and i4<I[4]:
                        μ_a_g = (bnd4[0] + bnd4[1]) / 2
                        μ_a_ps = np.array([μ_a_ng, μ_a_g, μ_a_ng, μ_a_g]).reshape(2,2)
                        
                        # Solve model given parameter values
                        bh = BlandinHerrington(λsp=λsp, μ_a_ps=μ_a_ps, loc_ξ_0=loc_ξ_0, ϕ=ϕ, ω_g=ω_g, α=α, ψ=ψ, ε=ε, σ=σ, σ_a=σ_a, ζ=ζ, σ_H=σ_H, NH=NH, NsdH=NsdH, scale_ξ=scale_ξ, θm=θm, ρ=ρ, path=path)
                        d_ps, d_agg, d_agg_sd, prob_a, hh, reg = run_parametrized_model(bh, path_data)

                        # Evaluate target 4 and adjust bnd4 without readjusting ϕ or μ_a_ng
                        err4 = d_ps['hh'][1, 1] - mn_hh_ps[1, 1] - 1
                        bnd4[0] = bnd4[0] * (err4>=-Err[4]) + μ_a_g * (err4<-Err[4])
                        bnd4[1] = μ_a_g * (err4>Err[4]) + bnd4[1] * (err4<=Err[4])
                        i4 += 1                        

                    # Evaluate target 3 and adjust bnd3
                    err3 = d_agg['sh_a'] - share_a
                    bnd3[0] = loc_ξ_0 * (err3>=-Err[3]) + bnd3[0] * (err3<-Err[3])
                    bnd3[1] = bnd3[1] * (err3>Err[3]) + loc_ξ_0 * (err3<=Err[3])
                    i3 += 1

                # Evaluate target 2 and adjust bnd2
                err2 = np.exp(d_agg['leph_g']) / np.exp(d_agg['leph_na']) - mn_prem
                bnd2[0] = bnd2[0] * (err2>Err[2]) + ω_g * (err2<=Err[2])
                bnd2[1] = ω_g * (err2>=-Err[2]) + bnd2[1] * (err2<-Err[2])  
                i2 += 1

            # Evaluate target 1 and adjust bnd1
            err1 = d_agg['i'] - mn_i
            bnd1[0] = bnd1[0] * (err1>Err[1]) + α * (err1<=Err[1])
            bnd1[1] = α * (err1>=-Err[1]) + bnd1[1] * (err1<-Err[1])        
            i1 += 1
            
        # Evaluate target 0 and adjust bnd0
        err0 = d_agg['m'] - mn_m - 3
        bnd0[0] = bnd0[0] * (err0>Err[0]) + θm * (err0<=Err[0])
        bnd0[1] = θm * (err0>=-Err[0]) + bnd0[1] * (err0<-Err[0])        
        i0 += 1
    
    # re-order output calibration summary for output to correspond with paper table
    descr =   ['Leisure utility weight', 'Parent altruism', 'CES weight of m', 'SD of parent human capital', 'College wage premium', 'Mean of taste shock', 'SD of taste shock', 'SD of idiosyncratic ability', 'Mean ability (non-college)', 'Mean ability (college)', 'Returns to scale', 'Exponent on H']
    params =  {0: 'psi',      1: 'alpha',    2: 'theta_m',  3: 'sigma_H',         4: 'omega_g / omega_ng',                               5: 'mu_zeta',     6: 'sigma_xi',        7: 'sigma_a',            8: 'mu_a_ng',                  9: 'mu_a_g',                   10: 'phi',            11: 'eta'}
    values =  {0: bh.ψ,       1: α,          2: θm,         3: bh.σ_H,            4: 1 + ω_g,                                            5: loc_ξ_0,       6: bh.scale_ξ,        7: bh.σ_a,               8: μ_a_ng,                     9: μ_a_g,                      10: ϕ,                11: bh.ζ }
    moments = ['Mean labor supply of adult children', 'Mean time investment', 'Mean market investment', 'Variance of log parent earnings', 'Coll/No Coll earn ratio', 'Aggregate attendance rate', 'corr(SAT, attendance)', 'SD of SAT within family types', 'Mean SAT, 2L', 'Mean SAT, 2H', 'Mean SAT, 1L', 'Coefficients in collete completion reg; See Table 5.']
    model =   {0: np.round(d_agg['n'], 2), 1: np.round(d_agg['i'], 3), 2: np.round(d_agg['m']), 3: np.round(d_agg_sd['lE']**2, 2), 4: np.round(np.exp(d_agg['leph_g']) / np.exp(d_agg['leph_na']), 2), 5: np.round(d_agg['sh_a'], 2), 6: np.round(corr_h_attend(bh), 2), 7: np.round(np.std(hh)), 8: np.round(d_ps['hh'][1, 0]), 9: np.round(d_ps['hh'][1, 1]), 10: np.round(d_ps['hh'][0, 0]), 11: ' ' }
    data =    {0: 0.40,       1: mn_i,       2: mn_m,       3: 0.554,             4: mn_prem,                                            5: share_a,       6: 0.40,              7: 91,                   8: mn_hh_ps[1, 0],             9: mn_hh_ps[1, 1],             10: mn_hh_ps[0, 0],   11: 0.09 }    
    
    # output calibration results
    if maketab==True:
        fpath = '../output/tables/table4.csv'
        rows = np.array([list(params.values()),
                descr,
                list(np.round(list(values.values()), 2)),
                moments,
                list(np.round(list(data.values()), 2)),
                list(model.values())
               ]).T
        with open(fpath, 'w', newline='') as f:
            pen = csv.writer(f)
            pen.writerow(['Parameter', 'Description', 'Value', 'Target Moment', 'Data', 'Model'])
            for row in rows:
                pen.writerow(list(row))

    return bh

def ctrf_alt_calib(nP, nS, bh, attend_target=np.array([[0.5523-0.070, 0.6724-0.273], [0.6187-0.155, 0.7590-0.020]])):
    
    ### Initialize parameters for calibration
    # Parameter order: θm, α, ω_g, loc_ξ_0, μ_a_g, ϕ, μ_a_ng
    # Target order: mn_m, mn_i, mn_prem, share_a, mn_hh_2h, mn_hh_2l, mn_hh_1l
    Err = 0.0005
    I = 20

    # calibrate value of loc_ξ_0
    
    bnd = [-2.0, 1.0]
    err, i = 1 + Err, 0
    while np.abs(err)>Err and i<I:
        loc_ξ_0 = (bnd[0] + bnd[1]) / 2

        # Solve model given parameter values
        bh.loc_ξ_0 = loc_ξ_0
        bh.loc_ξ = bh.loc_ξ_0 - bh.scale_ξ * 0.557
        d_ps, d_agg, d_agg_sd, prob_a, reg_gradprob_results = run_counterfactual(bh, path_d1995)

        # Evaluate target 3 and adjust bnd3
        err = d_ps['sh_a'][nP, nS] - attend_target[nP, nS]
        
        bnd[0] = loc_ξ_0 * (err>=-Err) + bnd[0] * (err<-Err)
        bnd[1] = bnd[1] * (err>Err) + loc_ξ_0 * (err<=Err)
        i += 1

    return bh

def shift_share_decomp(bh_early, bh_late):
    
    # Read stats from baseline and counterfactual models
    df0 = pd.read_csv(bh_early.path + 'statsvsdata.csv', skiprows=1)
    df1 = pd.read_csv(bh_late.path + 'statsvsdata.csv', skiprows=1)
    λ0 = np.array(df0.iloc[0, 2:6]).astype(float)
    λ0 = λ0 / np.sum(λ0)
    λ1 = np.array(df1.iloc[0, 2:6]).astype(float)
    λ1 = λ1 / np.sum(λ1)
    g_d = np.array(df0.iloc[5, 2:6]).astype(float)
    g_m = np.array(df0.iloc[13, 2:6]).astype(float)
    
    # compute baseline agg graduation
    g_d_agg0 = float(df0.iloc[5, 1])
    g_m_agg0 = float(df0.iloc[13, 1])
    g_d_agg1 = np.sum(λ1 * g_d)
    g_m_agg1 = np.sum(λ1 * g_m)
    
    # Write to new csv the difference btween counterfactual and baseline
    fpath = '../output/intext/intext_sec_V_C_shiftshare.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        pen.writerow(['share_effect_d', 'share_effect_m'])
        pen.writerow([g_d_agg1 - g_d_agg0, g_m_agg1 - g_m_agg0])
        
def corr_h_attend(bh, Nsim=10_000):
    """
    compute correlation between child human capital and college attendance
    """
    shape = (bh.NP, bh.NS, bh.NH, bh.Na, Nsim)
    attend, hh = np.zeros(shape), np.zeros(shape)

    prob_a, v2, v2_na, v2_a, y2_na, y2_a_ng, y2_a_g, c_na, c_a_ng, c_a_g, n_na, n_a_ng, n_a_g = bh.solve_c_2(h=bh.gridh, T=0)
    V2, Y2, C2, N2 = bh.solve_p_2()
    [mC, mN, mKK, mi, mm, mhh, mattend, mgrad, mn, mn_na, mn_ng, mn_g] = bh.solve_p_1(prob_a, v2, V2, C2, n_na, n_a_ng, n_a_g)

    for nP in range(bh.NP):
        for nS in range(bh.NS):
            for nH in range(bh.NH):
                for na in range(bh.Na):
                    for nsim in range(Nsim):
                        attend[nP, nS, nH, na, nsim] = np.random.binomial(n=1, p=mattend[nP, nS, nH, na])
                        hh[nP, nS, nH, na, nsim] = mhh[nP, nS, nH, na]
    
    return np.corrcoef(hh.flatten(), attend.flatten())[1, 0]

def maketables(datapath='', path=''):

    # table 6: counterfactual changes in investment, preparation, and college attainment
    dfm = pd.read_csv(datapath + '0/ctrf/stats_change.csv')
    dfi = pd.read_csv(datapath + '0/ctrf/investonly/stats_change.csv')
    rows = [[' ', ' ', ' ', 'Counterfactuals']]
    rows.append(['Variable', 'Family Type', 'Data', 'Main', 'Investment Only'])
    col_d = [0.018, 0.084, 0.111, 0.106, 0.070, 0.155, 0.020, 0.127, 106, 43, 429, 208, 1.75, 3.10, 4.41, 3.09, 13.3, 7.1]
    types = ['1L', '2L', '2H', 'Agg, Mn']
    vrbls = [7, 6, 4, 3]
    vlbls = ['Delta Completion share', 'Delta Attendance share', 'Delta Mean Market Investments', 'Delta Mean Time Investments', 'Delta Mean Human Capital Gap']
    rnd = {7: 3, 6: 3, 4: 0, 3: 2}
    col_v = [[vlbl, ' ', ' ', ' '] for vlbl in vlbls]
    col_v = [row for v in col_v for row in v][:-2]
    col_f = [f[:3] for nv in range(len(vrbls)) for f in types] + ['2L-1L', '2H-1L']
    col_m = [np.round(dfm[f][v], rnd[v]) for v in vrbls for f in types]
    col_m.extend(np.round([dfm['2L'][5] - dfm['1L'][5], dfm['2H'][5] - dfm['1L'][5]], 1))
    col_i = [np.round(dfi[f][v], rnd[v]) for v in vrbls for f in types]
    col_i.extend(np.round([dfi['2L'][5] - dfi['1L'][5], dfi['2H'][5] - dfi['1L'][5]], 1))
    cols = np.array([col_v, col_f, col_d, col_m, col_i]).T
    rows.extend(list(cols))
    fpath = path + 'tables/table6.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        for row in rows:
            pen.writerow(row)

    # table c1: alternative counterfactual changes in investment, preparation, and college attainment
    rows = [[' ', ' ', ' ', 'Counterfactuals']]
    rows.append(['Variable', 'Family Type', 'Data', 'Main', 'Alternative'])
    dfcal_level = pd.read_csv(datapath + '0/stats.csv')
    dfalt_level = {'1L': pd.read_csv(datapath + '0/ctrf/ctrf_alt/00/stats.csv'),
                   '2L': pd.read_csv(datapath + '0/ctrf/ctrf_alt/10/stats.csv'),
                   '1H': pd.read_csv(datapath + '0/ctrf/ctrf_alt/01/stats.csv'),
                   '2H': pd.read_csv(datapath + '0/ctrf/ctrf_alt/11/stats.csv')
                  }
    aggs = [np.round(dfcal_level['Agg, Mn'][v] - sum([dfalt_level[f][f][v] * dfalt_level[f][f][0] for f in dfalt_level.keys()]), rnd[v]) for nv, v in enumerate(vrbls)]
    dfalt = {'1L': pd.read_csv(datapath + '0/ctrf/ctrf_alt/00/stats_change.csv'),
             '2L': pd.read_csv(datapath + '0/ctrf/ctrf_alt/10/stats_change.csv'),
             '2H': pd.read_csv(datapath + '0/ctrf/ctrf_alt/11/stats_change.csv')
            }
    dfalt['Agg, Mn'] = dfalt['1L']
    col_alt = [np.round(dfalt[f][f][v], rnd[v]) for v in vrbls for f in types]
    col_alt[3], col_alt[7], col_alt[11], col_alt[15] = aggs
    col_alt.extend(np.round([dfalt['2L']['2L'][5] - dfalt['1L']['1L'][5], dfalt['2H']['2H'][5] - dfalt['1L']['1L'][5]], 1))
    cols = np.array([col_v, col_f, col_d, col_m, col_alt]).T
    rows.extend(list(cols))
    fpath = path + 'tables/tablec1.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        for row in rows:
            pen.writerow(row)

    # table c2: changes in investment and completion for different substitution elasticities
    vrbls = [7, 4, 3]
    vlbls = ['Delta Completion share', 'Delta Mean Market Investments', 'Delta Mean Time Investments']
    rows = [['Variable', 'Family Type', 'rho=-0.9', 'rho-->0 (Baseline)', 'rho=0.7']]
    col_v = [[vlbl, ' ', ' ', ' '] for vlbl in vlbls]
    col_v = [row for v in col_v for row in v]
    col_f = [f[:3] for nv in range(len(vrbls)) for f in types]
    col_m = [np.round(dfm[f][v], rnd[v]) for v in vrbls for f in types]
    dfsubs = pd.read_csv(datapath + '1/ctrf/stats_change.csv')
    col_subs = [np.round(dfsubs[f][v], rnd[v]) for v in vrbls for f in types]
    dfcomp = pd.read_csv(datapath + '2/ctrf/stats_change.csv')
    col_comp = [np.round(dfcomp[f][v], rnd[v]) for v in vrbls for f in types]
    cols = np.array([col_v, col_f, col_comp, col_m, col_subs]).T
    rows.extend(list(cols))
    fpath = path + 'tables/tablec2.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        for row in rows:
            pen.writerow(row)

    # in-text numbers for section V.B
    df = pd.read_csv(datapath + '0/ctrf/stats_change.csv')
    rows = [['Family Type', '1L', '2L', '2H']]
    rows.append(['Delta human capital', df['1L'][5], df['2L'][5], df['2H'][5]])
    fpath = path + 'intext/intext_sec_V_B_changehc.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        for row in rows:
            pen.writerow(row)
    
    
    # in-text numbers for section V.D
    df = {}
    df[0] = pd.read_csv(datapath + '0/ctrf/stats_change.csv')
    df[1] = pd.read_csv(datapath + '0/ctrf/decomp_1/stats_change.csv')
    df[2] = pd.read_csv(datapath + '0/ctrf/decomp_2/stats_change.csv')
    df[3] = pd.read_csv(datapath + '0/ctrf/decomp_3/stats_change.csv')
    rows = [['Unit', 'Delta sd: overall', 'Delta sd: premium', 'Delta sd: composition', 'Delta sd: preparation']]
    rows.append(['Parent', df[0]['Agg, SD'][1], df[1]['Agg, SD'][1], df[2]['Agg, SD'][1], ' '])
    rows.append(['Child', df[0]['Agg, SD'][12], df[2]['Agg, SD'][12], ' ', df[3]['Agg, SD'][12]])
    fpath = path + 'intext/intext_sec_V_D_inequality.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        for row in rows:
            pen.writerow(row)

    # table 7: policy experiments: average effects among 1L children
    df = {}
    df[0] = pd.read_csv(datapath + '0/stats.csv')
    df[1] = pd.read_csv(datapath + '0/transfer_tuition/stats.csv')
    df[2] = pd.read_csv(datapath + '0/transfer/stats.csv')
    df[3] = pd.read_csv(datapath + '0/transfer_m/stats.csv')
    df[4] = pd.read_csv(datapath + '0/subsidy_m/stats.csv')
    df[5] = pd.read_csv(datapath + '0/subsidy_i/stats.csv')
    rows = np.zeros([4, 5])
    vrbls = [5, 6, 7, 13]
    vlbls = ['Delta h (SAT points)', 'Delta attendance (pp)', 'Delta completion (pp)', 'Delta mean lifetime earnings net of tuition (%)']
    scale = {5: 1, 6: 100, 7: 100, 13: 100}
    for m in range(1, 6):
        for nv, v in enumerate(vrbls):
            rows[nv, m - 1] = np.round(scale[v] * (df[m]['1L'][v] - df[0]['1L'][v]), 1)
    fpath = path + 'tables/table7.csv'
    with open(fpath, 'w', newline='') as f:
        pen = csv.writer(f)
        pen.writerow(['Change relative to baseline model', 'Tuition Subsidy', 'Unrestricted Transfer', 'Restricted Transfer', 'Subsidy for m', 'Subsidy for i'])
        for row, vlbl in zip(rows, vlbls):
            pen.writerow([vlbl] + list(row))

########################################################################################

##########################
##  Create directories  ##
##########################

from pathlib import Path
for nρ in range(3):
    s = str(nρ)
    
    for f in ['00', '01', '10', '11']:
        
        Path('../intermediate/' + s + '/ctrf/ctrf_alt/' + f).mkdir(parents=True, exist_ok=True)
    
    for f in ['decomp_1', 'decomp_2', 'decomp_3', 'investonly']:
        
        Path('../intermediate/' + s + '/ctrf/' + f).mkdir(parents=True, exist_ok=True)
    
    for f in ['subsidy_i', 'subsidy_m', 'transfer', 'transfer_m', 'transfer_tuition']:
        
        Path('../intermediate/' + s + '/' + f).mkdir(parents=True, exist_ok=True)


            
########################################################################################

#########################
##  Calibrate to 2015  ##
#########################

# Set empirical targets for calibration of baseline model
mn_m, mn_i, mn_prem, sd_hh = 488, 0.055, 1.80, 85, 
share_a, share_a_ps = 0.632, np.array([0.45, 0.77, 0.64, 0.84]).reshape(2, 2)
mn_hh, mn_hh_ps = 467.7, np.array([422, 484, 449, 524]).reshape(2, 2)
estD = [.092, -.076, .107, .318]
path_d1995, path_d2015 = '../data/moments_1995_adj_forleis.csv', '../data/moments_2015_adj_forleis.csv'

# parameters for baseline model
λsp = np.array([0.32, 0.04, 0.40, 0.24]).reshape(2,2)
scale_ξ = 0.40
σ_a = 0.175
ζ = 0.098
σ_H, NH, NsdH = 0.55, 5, 3
ε, ψ, σ = 1.01, 1.65, 2.00001

for nρ, ρ in enumerate([0.01, 0.7, -0.9]):
    print('')
    print('nρ = ', nρ, ', ρ = ', np.round(ρ, 2))
    print('')
    maketab = True if nρ==0 else False
    
    ###########################################################
    ##    baseline calibration 
    ###########################################################
    bh = calibrate(mn_m, mn_i, mn_prem, sd_hh, share_a, share_a_ps, mn_hh, mn_hh_ps, λsp, σ=σ, ε=ε, ψ=ψ, ρ=ρ, σ_a=σ_a, ζ=ζ, σ_H=σ_H, NH=NH, NsdH=NsdH, scale_ξ=scale_ξ, maketab=maketab, path='../intermediate/{}/'.format(nρ), path_data=path_d2015)
    μ_a_ps, loc_ξ_0, σ_a, ω_g, α, ψ, ε = bh.μ_a_ps, bh.loc_ξ_0, bh.σ_a, bh.ω_g, bh.α, bh.ψ, bh.ε
    ϕ, scale_ξ, θm, ρ = bh.ϕ, bh.scale_ξ, bh.θm, bh.ρ
    τ_m_ps = bh.τ_m_ps
    bh = BlandinHerrington(λsp=λsp, ε=ε, ψ=ψ, ω_g=ω_g, α=α, ϕ=ϕ, loc_ξ_0=loc_ξ_0, μ_a_ps=μ_a_ps, σ=σ, σ_a=σ_a, ζ=ζ, σ_H=σ_H, NH=NH, NsdH=NsdH, scale_ξ=scale_ξ, θm=θm, ρ=ρ, path='../intermediate/{}/'.format(nρ))   
    
    # Compute statistics for calibrated (baseline) model
    d_ps, d_agg, d_agg_sd, prob_a_fixed, hh, reg = run_parametrized_model(bh, path_d2015, output=True)
    
    if maketab==True:
        bh.maketab5(reg)
    
    ###########################################################
    ##    main and investment-only counterfactuals  
    ###########################################################
    
    # Set exogenous parameters for main counterfactual
    λsp_2 = np.array([0.28, 0.02, 0.59, 0.11]).reshape(2,2) 
    ω_g_2 = 0.41 
    τ_m_2 = 4 * np.array([3_345, 5_657, 2_534, 4_846]).reshape(2,2)

    # Compute statistics for main counterfactual
    bh_2 = BlandinHerrington(λsp=λsp_2, ε=ε, ψ=ψ, τ_m_ps=τ_m_2, ω_g=ω_g_2, α=α, ϕ=ϕ, σ=σ, loc_ξ_0=loc_ξ_0, μ_a_ps=μ_a_ps, σ_a=σ_a, ζ=ζ, σ_H=σ_H, NH=NH, NsdH=NsdH, scale_ξ=scale_ξ, θm=θm, ρ=ρ, path='../intermediate/{}/ctrf/'.format(nρ))
    d_ps_2, d_agg_2, d_agg_sd_2, prob_a_2, reg_gradprob_results = run_counterfactual(bh_2, path_d1995)
    bh_2.output_results(d_ps_2, d_agg_2, d_agg_sd_2, prob_a_2, reg_gradprob_results, path_d1995)
    compare_baseline_ctrf(bh_early=bh_2, bh_late=bh) # Compute change in statistics between counterfactual and calibrated
    shift_share_decomp(bh_early=bh_2, bh_late=bh)
        
    # Compute statistics for investment-only counterfactual
    bh_2.path = bh_2.path + 'investonly/'
    d_ps_2i, d_agg_2i, d_agg_sd_2i, prob_a_2i, reg_gradprob_results = run_counterfactual(bh_2, path_data=path_d1995, invonly=True, prob_a_fixed=prob_a_fixed)
    bh_2.output_results(d_ps_2i, d_agg_2i, d_agg_sd_2i, prob_a_2i, reg_gradprob_results, path_d1995)
    compare_baseline_ctrf(bh_early=bh_2, bh_late=bh) # Compute change in statistics between counterfactual and calibrated
    
    
    ###########################################################
    ##    inequality decomposition counterfactuals  
    ###########################################################   
    
    # role of college premium
    path_ctrf = '../intermediate/0/ctrf/decomp_1/'
    bh_ctrf = decompose_ineq(bh, ω_g_2, λsp, path_ctrf, path_d1995)
    compare_baseline_ctrf(bh_early=bh_ctrf, bh_late=bh) 

    # role of family composition
    path_ctrf = '../intermediate/0/ctrf/decomp_2/'
    bh_ctrf = decompose_ineq(bh, ω_g, λsp_2, path_ctrf, path_d1995)
    compare_baseline_ctrf(bh_early=bh_ctrf, bh_late=bh)  

    # role of changing preparedness
    path_ctrf = '../intermediate/0/ctrf/decomp_3/'
    bh_ctrf = decompose_ineq(bh_2, ω_g, λsp, path_ctrf, path_d1995)
    compare_baseline_ctrf(bh_early=bh_ctrf, bh_late=bh) 
    
    
    ###########################################################
    ##    alternative counterfactual: "target attendance"    
    ###########################################################

    # iterate through family types
    for nP in range(2):
        for nS in range(2):

            # calibrate new shock mean for type
            bh_2 = BlandinHerrington(λsp=λsp_2, ε=ε, ψ=ψ, τ_m_ps=τ_m_2, ω_g=ω_g_2, α=α, ϕ=ϕ, σ=σ, loc_ξ_0=loc_ξ_0, μ_a_ps=μ_a_ps, σ_a=σ_a, ζ=ζ, σ_H=σ_H, NH=NH, NsdH=NsdH, scale_ξ=scale_ξ, θm=θm, ρ=ρ, path='../intermediate/{}/ctrf/'.format(nρ))
            bh_2alt = ctrf_alt_calib(nP, nS, bh_2)
            bh_2alt.path='../intermediate/{}/ctrf/ctrf_alt/{}{}/'.format(nρ, nP, nS)

            # Compute statistics for main counterfactual
            d_ps_2alt, d_agg_2alt, d_agg_sd_2alt, prob_a_2alt, reg_gradprob_results = run_counterfactual(bh_2alt, path_d1995)
            bh_2alt.output_results(d_ps_2alt, d_agg_2alt, d_agg_sd_2alt, prob_a_2alt, reg_gradprob_results, path_d1995)

            # Compute change in statistics between counterfactual and calibrated
            compare_baseline_ctrf(bh_early=bh_2alt, bh_late=bh)    
    
    
    
    ###########################################################
    ##    policy counterfactuals targeted to 1L families  
    ###########################################################
    
    # Compute tuition subsidy
    bh_3 = deepcopy(bh)
    bh_3.path='../intermediate/{}/transfer_tuition/'.format(nρ)
    Tτ = counterfactual_tuitionsubsidy(bh_3, path_data=path_d2015, attend_2L=.605, minTτ=τ_m_ps[0, 0], maxTτ=τ_m_ps[0, 0], NTτ=1)
    
    # Compute i subsidy
    bh_4 = deepcopy(bh)
    bh_4.path='../intermediate/{}/subsidy_i/'.format(nρ)
    counterfactual_isubsidy(bh_4, path_data=path_d2015, T=Tτ * .571)        
    
    # Compute m subsidy
    bh_5 = deepcopy(bh)
    bh_5.path='../intermediate/{}/subsidy_m/'.format(nρ)
    counterfactual_msubsidy(bh_5, path_data=path_d2015, T=Tτ * .571)  
    
    # Compute m transfer
    bh_6 = deepcopy(bh)
    bh_6.path='../intermediate/{}/transfer_m/'.format(nρ)
    counterfactual_mtransfer(bh_6, path_data=path_d2015, attend_2L=.605, minTm=Tτ * .571, maxTm=Tτ * .571, NTm=1)        
    
    # Compute m transfer
    bh_7 = deepcopy(bh)
    bh_7.path='../intermediate/{}/transfer/'.format(nρ)
    counterfactual_transfer(bh_7, path_data=path_d2015, T=Tτ * .571) 

maketables(datapath='../intermediate/', path='../output/')

ERROR: Could not find a version that satisfies the requirement anaconda-navigator==1.9.7
ERROR: No matching distribution found for anaconda-navigator==1.9.7



nρ =  0 , ρ =  0.01



<ipython-input-1-d4277c7b503d>:129: RuntimeWarning: invalid value encountered in double_scalars
  return vr**0.5, vr_PS**0.5
<ipython-input-1-d4277c7b503d>:129: RuntimeWarning: invalid value encountered in sqrt
  return vr**0.5, vr_PS**0.5



nρ =  1 , ρ =  0.7


nρ =  2 , ρ =  -0.9

